In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

df = pd.read_csv('/Users/adamaskari/Downloads/data_house.csv.zip')
df = df.drop(['yr_renovated', 'long', 'lat', 'date', 'Unnamed: 0', 'id'], axis=1)
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,98178,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,98125,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,98028,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,98136,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,98074,1800,7503


In [6]:
df['mean_zipcode_price'] = df.groupby('zipcode')['price'].transform('mean')
print(df)

          price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  \
0      221900.0         3       1.00         1180      5650     1.0   
1      538000.0         3       2.25         2570      7242     2.0   
2      180000.0         2       1.00          770     10000     1.0   
3      604000.0         4       3.00         1960      5000     1.0   
4      510000.0         3       2.00         1680      8080     1.0   
...         ...       ...        ...          ...       ...     ...   
21608  360000.0         3       2.50         1530      1131     3.0   
21609  400000.0         4       2.50         2310      5813     2.0   
21610  402101.0         2       0.75         1020      1350     2.0   
21611  400000.0         3       2.50         1600      2388     2.0   
21612  325000.0         2       0.75         1020      1076     2.0   

       waterfront  view  condition  grade  sqft_above  sqft_basement  \
0               0     0          3      7        1180              0   
1  

In [7]:
features = df.drop(columns = ["price"])
target = df["price"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

Q1 = df["price"].quantile(0.25)
Q3 = df["price"].quantile(0.75)
IQR = Q3 - Q1

mask = (df["price"] >= (Q1 - 1.5 * IQR)) & (df["price"] <= (Q3 + 1.5 * IQR))

df = df[mask]

In [8]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [9]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [10]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,sqft_living15,sqft_lot15,mean_zipcode_price
0,0.090909,0.1250,0.096604,0.003023,0.0,0.0,0.0,0.75,0.500000,0.085526,0.103734,0.347826,0.575758,0.240063,0.006581,0.200183
1,0.090909,0.3125,0.112453,0.006348,0.0,0.0,0.0,0.75,0.583333,0.100877,0.118257,0.695652,0.025253,0.334617,0.010395,0.324660
2,0.090909,0.1875,0.060377,0.005659,0.0,0.0,0.0,0.50,0.583333,0.087719,0.000000,0.756522,0.368687,0.229557,0.010581,0.234292
3,0.121212,0.3125,0.144906,0.003973,0.4,0.0,0.0,0.50,0.583333,0.210526,0.000000,0.808696,0.151515,0.275083,0.007293,0.034395
4,0.090909,0.3125,0.113962,0.002570,0.4,0.0,0.0,0.50,0.500000,0.104167,0.116183,0.739130,0.595960,0.243565,0.004723,0.319346


In [11]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
X_test_norm.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,sqft_living15,sqft_lot15,mean_zipcode_price
0,0.060606,0.18750,0.086038,0.000685,0.8,0.0,0.0,0.50,0.500000,0.125000,0.000000,0.860870,0.626263,0.180529,0.001148,0.122083
1,0.121212,0.40625,0.330566,0.031085,0.4,0.0,0.0,0.75,0.916667,0.480263,0.000000,0.765217,0.020202,0.670811,0.046435,0.298953
2,0.060606,0.09375,0.086792,0.001926,0.0,0.0,0.0,0.50,0.500000,0.099781,0.049793,0.121739,0.535354,0.182280,0.004192,0.178978
3,0.060606,0.12500,0.063396,0.001284,0.0,0.0,0.0,0.75,0.583333,0.092105,0.000000,0.234783,0.545455,0.224304,0.002928,0.335011
4,0.121212,0.31250,0.218113,0.005502,0.4,0.0,0.5,0.50,0.666667,0.316886,0.000000,0.886957,0.777778,0.357380,0.016783,0.098344


In [12]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)